reqMktData() to get bid-ask data is sucessful

In [10]:
from ib.opt import ibConnection, message
from ib.ext.Contract import Contract
from ib.ext.TickType import TickType as tt
from time import sleep
from __future__ import print_function #I'm using 3.x style print
import pandas as pd

In [11]:
def error_handler(msg):
    print (msg)

In [12]:
def my_callback_handler(msg):
    inside_mkt_bid = ''
    inside_mkt_ask = ''
    
    if msg.field == 1:
        inside_mkt_bid = msg.price
        print ("bid", inside_mkt_bid)
    elif msg.field == 2:
        inside_mkt_ask = msg.price
        print ('ask', inside_mkt_ask)

In [13]:
def my_callback_handler(msg):
    if msg.field in [tt.BID,tt.ASK,tt.LAST]:
        #now we can just store the response in the data frame
        contracts.loc[msg.tickerId,tt.getField(msg.field)] = msg.price
        if msg.field == tt.LAST:
            print(contracts.loc[msg.tickerId,'sym'],msg.price)

In [14]:
#tws = ibConnection()
tws = ibConnection(clientId=1)
tws.register(my_callback_handler, message.tickPrice, message.tickSize)
tws.register(error_handler, 'Error')
tws.connect()

Server Version: 76
TWS Time at connection:20160921 21:57:22 IST


True

<error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:usfarm.us>
<error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:hfarm>
<error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:cashfarm>
<error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:usfuture>
<error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:usbond>
<error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:jfarm>
<error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:eufarm>
<error id=-1, errorCode=2104, errorMsg=Market data farm connection is OK:usfarm>
<error id=-1, errorCode=2106, errorMsg=HMDS data farm connection is OK:ilhmds>
<error id=-1, errorCode=2106, errorMsg=HMDS data farm connection is OK:euhmds>
<error id=-1, errorCode=2106, errorMsg=HMDS data farm connection is OK:fundfarm>
<error id=-1, errorCode=2106, errorMsg=HMDS data farm connection is OK:ushmds>


In [15]:
c = Contract()
c.m_symbol = "VIX"
c.m_secType = "IND"
c.m_exchange = "CBOE"
c.m_currency = "USD"

In [16]:
tws.reqMktData(1,c,"",False)
sleep(1)

In [17]:
tws.disconnect()

True

ANOTHER 

http://stackoverflow.com/questions/30130319/ibpy-extract-api-responces-for-multiple-contracts

ANOTHER

In [12]:
%run ibpy_marketdata.py

Server Version: 76
TWS Time at connection:20160919 20:45:49 IST
Contract Values:SPY,STK,SMART,USD,,0.0,:
* * * * REQUESTING MARKET DATA * * * *
SPY:: bid: 214.35
SPY:: ask: 214.36
SPY:: bid: 214.37
SPY:: ask: 214.38
SPY:: bid: 214.38
SPY:: ask: 214.39
SPY:: ask: 214.4
SPY:: bid: 214.39
SPY:: bid: 214.4
SPY:: ask: 214.41
SPY:: bid: 214.41
SPY:: ask: 214.42
SPY:: bid: 214.43
SPY:: ask: 214.44
SPY:: bid: 214.44
SPY:: ask: 214.45
SPY:: bid: 214.42
SPY:: ask: 214.43
SPY:: bid: 214.43
SPY:: ask: 214.44
SPY:: bid: 214.44
SPY:: ask: 214.45
SPY:: bid: 214.45
SPY:: ask: 214.46
SPY:: bid: 214.46
SPY:: ask: 214.47
SPY:: bid: 214.45
SPY:: ask: 214.46
SPY:: bid: 214.46
SPY:: ask: 214.47
SPY:: bid: 214.45
SPY:: ask: 214.46
SPY:: bid: 214.46
SPY:: ask: 214.47


In [14]:
print '* * * * CANCELING MARKET DATA * * * *'
con.cancelMktData(tickId)
sleep(1)
print '* * * * Disconnecting * * * *'
con.disconnect()
sleep(1)

* * * * CANCELING MARKET DATA * * * *
* * * * Disconnecting * * * *


Another

In [ ]:
from ib.ext.Contract import Contract
from ib.opt import ibConnection, message
from Queue import Queue, Empty
from time import time, sleep
from thread import start_new_thread

In [ ]:
#symbols = ["AUDUSD","EURUSD","GBPUSD","USDCAD","USDCHF","USDJPY","GBPJPY","EURJPY"]
symbols = ["QQQ","SPY","SQQQ"]
q = [None] * len(symbols)

In [ ]:
def writer_thread_func(symbol_index):
    global symbols, q
    symbol = symbols[symbol_index]

    while True:
        try:
            (time,price) = q[symbol_index].get(True,15)
            print symbol, time, price
            # Other processing with the data that might take some time.
            # ...
        except Empty:
            print "no data for 15 seconds"
            # Other processing for when 15 seconds elapsed without any data.
            # ...

In [ ]:
def my_callback(msg):
    global symbols, q
    if isinstance(msg,message.TickPrice):
        q[msg.tickerId].put_nowait((time(),msg.price))
    #elif:
        # other message types etc.

In [ ]:
def make_contract(symbol,sectype,exchange,currency,expiry,strike,right):
    cont = Contract()
    cont.m_symbol = symbol
    cont.m_secType = sectype
    cont.m_exchange = exchange
    cont.m_currency = currency
    cont.m_expiry = expiry
    cont.m_strike = strike
    cont.m_right = right
    return cont

In [ ]:
con = ibConnection('127.0.0.1', 7496, 123)
con.registerAll(my_callback)
con.connect()

In [ ]:
req_id = 0
for symbol in symbols:

    q[req_id] = Queue(0)
    start_new_thread(writer_thread_func,(req_id,))
    symbol_contract = make_contract(symbol[0:3], 'STK', 'IDEALPRO', symbol[3:6], '', 0.0, '')
    con.reqMktData(req_id, symbol_contract, '', False)
    req_id += 1

#sleep(-1)

In [ ]:
con.disconnect()

ANother

In [7]:
import re
import ib
from ib.ext.Contract import Contract
from ib.opt import ibConnection, message
from time import sleep

In [8]:
class Downloader(object):
    field4price = ''

    def __init__(self):
        #self.tws = ibConnection('localhost',7496,123)
        self.tws = ibConnection('localhost',7496,123)
        self.tws.register(self.tickPriceHandler, 'TickPrice')
        self.tws.connect()
        self._reqId = 1 # current request id

    def tickPriceHandler(self,msg):
        if msg.field == 4:
            self.field4price = msg.price
            #print '[debug]', msg

    def requestData(self,contract): 
        self.tws.reqMktData(self._reqId, contract, '', 1)
        self._reqId+=1

In [9]:
#if __name__=='__main__':
dl = Downloader()
c = Contract()
c.m_symbol = 'SPY'
c.m_secType = 'STK'
c.m_exchange = 'SMART'
c.m_currency = 'USD'
dl.requestData(c)
sleep(0.01)
print 'Price - field 4: ', dl.field4price

Server Version: 76
TWS Time at connection:20160921 19:49:01 IST
Price - field 4:  


https://github.com/mementum/backtrader/issues/69